In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install datasets
!pip install seqeval

     |████████████████████████████████| 4.0 MB 14.0 MB/s 
     |████████████████████████████████| 596 kB 58.7 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 895 kB 50.6 MB/s 
     |████████████████████████████████| 6.6 MB 53.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 13.4 MB/s 
     |████████████████████████████████| 212 kB 65.7 MB/s 
     |████████████████████████████████| 136 kB 83.1 MB/s 
     |████████████████████████████████| 1.1 MB 69.8 MB/s 
     |████████████████████████████████| 127 kB 92.9 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 271 kB 75.7 MB/s 
     |████████████████████████████████| 144 kB 65.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [3]:
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

In [4]:
label_list = ['I-LOC', 'E-PER', 'O', 'S-OBJ', 'E-OBJ', 'E-ORG', 'I-ORG', 'S-PER', 'I-PER', 'B-PER', 'B-ORG', 'B-LOC', 'S-ORG', 'I-OBJ', 'B-OBJ', 'S-LOC', 'E-LOC']
# label_encoding_dict = {'B-LOC': 0, 'E-LOC': 1, 'S-ORG': 2, 'S-OBJ': 3, 'I-OBJ': 4, 'S-PER': 5, 'E-PER': 6, 'B-PER': 7, 'B-ORG': 8, 'S-LOC': 9, 'I-PER': 10, 'I-LOC': 11, 'B-OBJ': 12, 'E-ORG': 13, 'O':14, 'I-ORG': 15, 'E-OBJ': 16}
label_encoding_dict = {'I-PRG': 2,'I-I-MISC': 2, 'I-OR': 6, 'O': 0, 'I-': 0, 'VMISC': 0, 'B-PER': 3, 'I-PER': 4, 'B-ORG': 5, 'I-ORG': 6, 'B-LOC': 7, 'I-LOC': 8, 'B-MISC': 1, 'I-MISC': 2, 'E-PER': 9, 'S-OBJ': 10, 'E-OBJ': 11, 'E-ORG': 12, 'S-PER': 13, 'S-ORG': 14, 'I-OBJ': 15, 'B-OBJ': 16, 'S-LOC': 17, 'E-LOC': 18}

In [5]:
task = "ner" 
model_checkpoint = "KamrusSamad/tiny_A-2_H-2"
batch_size = 16
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [6]:
# train = pd.read_csv("/content/drive/MyDrive/Datasets/NER/train_ner.csv")
# test = pd.read_csv("/content/drive/MyDrive/Datasets/NER/test_ner.csv")

# train.head()

In [7]:
# train_texts = train['word'].values.tolist()
# train_labels = train['bioes_tags'].values.tolist()
# train_labels2 = train['iob_tags'].values.tolist()

# test_texts = test['word'].values.tolist()
# test_labels = test['bioes_tags'].values.tolist()
# test_labels2 = test['iob_tags'].values.tolist()

In [8]:
# traind = pd.DataFrame({'tokens': train_texts, 'ner_tags': train_labels})
# testd = pd.DataFrame({'tokens': test_texts, 'ner_tags': test_labels})

In [9]:
# traind

In [10]:
# train_dataset = Dataset.from_pandas(traind)
# test_dataset = Dataset.from_pandas(testd)

In [35]:
def get_all_tokens_and_ner_tags(directory):
    return pd.concat([get_tokens_and_ner_tags(os.path.join(directory, filename)) for filename in os.listdir(directory)]).reset_index().drop('index', axis=1)
    
def get_tokens_and_ner_tags(filename):
    with open(filename, 'r', encoding="utf8") as f:
        lines = f.readlines()
        split_list = [list(y) for x, y in itertools.groupby(lines, lambda z: z == '\n') if not x]
        tokens = [[x.split('\t')[0] for x in y] for y in split_list]
        # print(tokens[:30])
        entities = [[x.split('\t')[1][:-1] for x in y] for y in split_list]
        # print(entities[:30]) 
    return pd.DataFrame({'tokens': tokens, 'ner_tags': entities})
  
def get_un_token_dataset(train_directory, test_directory):
    train_df = get_all_tokens_and_ner_tags(train_directory)
    test_df = get_all_tokens_and_ner_tags(test_directory)
    print(test_df.head())
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    return (train_dataset, test_dataset)

In [36]:
train_dataset, test_dataset = get_un_token_dataset('/content/drive/MyDrive/Datasets/NER/train/', '/content/drive/MyDrive/Datasets/NER/test/')

                                              tokens  \
0  [৫%, তার, চাইতে, পশ্চিমোরে, এর, সাক্ষরতার, হার...   

                                            ner_tags  
0  [O, O, O, S-LOC, O, O, O, O, O, O, O, O, O, O,...  


In [14]:
train_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 1
})

In [19]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        print(label[:30])
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

['O', 'O', 'O', 'S-PER', 'B-PER', 'I-PER', 'E-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-PER', 'S-OBJ', 'O', 'O', 'S-PER', 'O', 'O', 'S-PER', 'O', 'O', 'S-PER', 'O', 'S-OBJ', 'O', 'O', 'O']


  0%|          | 0/1 [00:00<?, ?ba/s]

['O', 'O', 'O', 'S-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'E-PER', 'O', 'S-OBJ', 'O', 'O', 'S-PER', 'S-PER', 'O', 'O', 'S-LOC']


In [20]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

Some weights of the model checkpoint at KamrusSamad/tiny_A-2_H-2 were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at KamrusSamad/tiny_A-2_H-2 and are

In [21]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1


RuntimeError: ignored

In [ ]:
trainer.evaluate()